[Open PDF](problem-pokemon.pdf)

Pokemones Total: 900
Equipo Fijo: 30 rand 
Equipo Eval: 30

Evaluar según:
* damage_relations con otros types
* exp

In [7]:
import requests
import random
random.seed(42)

In [37]:
import requests
import random
random.seed(42)

class PokemonBattle:
  class PokemonClass:
    def __init__ (self, idx, pokeApi, typeApi, damage):
      self.idx = idx
      self.pokeApi = pokeApi
      self.typeApi = typeApi
      self.damage = damage

  def __init__(self, api_url):
    self.api_url = api_url
    self.totalPokemons = []
    self.evaluatedTeam = []
    self.fixedTeam = []
    self.typesDictFixedTeam = dict()
    self.randomIdx = random.sample(range(1,500), 30)
    self.TOTAL_TEAM = 10

  def getTypes(self, idx):
    try:
      responseInfo = requests.get(f"{self.api_url}/{idx}")
      responseInfo.raise_for_status()  # Raise an exception for 4XX or 5XX status codes
      data = responseInfo.json()
      poke_type = data['types'][0]['type'] # Always first type
      responseTypeUrl = requests.get(poke_type['url'])
      responseTypeUrl.raise_for_status()  # Raise an exception for 4XX or 5XX status codes
      data = responseTypeUrl.json()
      damage = data['damage_relations']
      for key, values in damage.items():
        damage[key] = [item["url"] for item in values]
      return poke_type, damage
    except requests.exceptions.RequestException as e:
      print(f"Error fetching data from API: {e}")
      return None, None

  def getPokemons(self):
    try:
      response = requests.get(f"{self.api_url}?limit=900")
      response.raise_for_status()  # Raise an exception for 4XX or 5XX status codes
      data = response.json()
      self.totalPokemons = data['results']
      for idx in self.randomIdx:
        poke_type, damage = self.getTypes(idx)
        if poke_type:
          if poke_type['url'] not in self.typesDictFixedTeam:
            self.typesDictFixedTeam[poke_type['url']] = 1
          else:
            self.typesDictFixedTeam[poke_type['url']] += 1
          self.fixedTeam.append(self.PokemonClass(idx, self.totalPokemons[idx], poke_type, damage))
    except requests.exceptions.RequestException as e:
      print(f"Error fetching data from API: {e}")

  def totalDamage(self, typesFixed, poke_damage):
    total_damage = 0
    if typesFixed in poke_damage['double_damage_from']:
      total_damage -= 2
    elif typesFixed in poke_damage['double_damage_to']:
      total_damage += 2
    elif typesFixed in poke_damage['half_damage_from']:
      total_damage -= 1
    elif typesFixed in poke_damage['half_damage_to']:
      total_damage += 1
    elif typesFixed in poke_damage['no_damage_from']:
      total_damage += 50
    elif typesFixed in poke_damage['no_damage_to']:
      total_damage -= 50
    return total_damage

  def evaluateTeam(self):
    self.getPokemons()
    try:
      generatedIdx = set()
      while(len(self.evaluatedTeam) < self.TOTAL_TEAM):
        randomIdx = random.randint(1,500)
        if randomIdx not in generatedIdx:
          generatedIdx.add(randomIdx)
          poke_type, poke_damage = self.getTypes(randomIdx)
          if poke_type['url']:
            total_relation = 0
            for fixed_type, amount in self.typesDictFixedTeam.items(): #Uniques Types
              total_damage = self.totalDamage(fixed_type, poke_damage)
              total_relation += total_damage * amount
            if total_relation > 0:
              # print(total_relation)
              self.evaluatedTeam.append(self.PokemonClass(randomIdx, self.totalPokemons[randomIdx], poke_type['url'], poke_damage))
      team_names = [self.evaluatedTeam[i].pokeApi['name'] for i in range(len(self.evaluatedTeam))]
      fixed_team_names = [self.fixedTeam[i].pokeApi['name'] for i in range(len(self.fixedTeam))]
      return self.evaluatedTeam, team_names, fixed_team_names
    except requests.exceptions.RequestException as e:
      print(f"Error fetching data from API: {e}")

api_url = "https://pokeapi.co/api/v2/pokemon"
pokemon_api = PokemonBattle(api_url)
_ , team_names, fixed_team_names = pokemon_api.evaluateTeam()

In [38]:
team_names, fixed_team_names

(['tangela',
  'articuno',
  'vespiquen',
  'riolu',
  'chimchar',
  'mothim',
  'farfetchd',
  'slugma',
  'togetic',
  'igglybuff'],
 ['vibrava',
  'arcanine',
  'kakuna',
  'latios',
  'aerodactyl',
  'pinsir',
  'horsea',
  'tentacruel',
  'registeel',
  'psyduck',
  'armaldo',
  'mantyke',
  'kirlia',
  'paras',
  'aron',
  'slugma',
  'pidgeot',
  'pidgeotto',
  'venomoth',
  'chansey',
  'starmie',
  'swampert',
  'manectric',
  'beedrill',
  'slaking',
  'exeggutor',
  'gorebyss',
  'altaria',
  'snorunt',
  'teddiursa'])